# **🏠 부동산 실거래가 competition**
> 대회의 기본 데이터를 확인하고 어떤 전략을 세울지 정하는 페이지입니다.
> 제일 처음 세웠던 기초 전략과 함께 저장했습니다.

## Contents
- Library Import
- Data Load
- Data Imputation


## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [1]:
# !pip install eli5==0.13.0

# # 한글 폰트 사용을 위한 라이브러리입니다.
# !apt-get install -y fonts-nanum

In [5]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import geopy
from geopy.geocoders import Nominatim

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

## 2. Data Load

#### 2.1. 데이터 로드

In [6]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/train.csv'
test_path  = '../data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

In [7]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)


In [8]:
# Train과 Test data를 살펴보겠습니다.
display(dt.head(1))
display(dt_test.head(1))      # 부동산 실거래가(=Target) column이 제외된 모습입니다.

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,...,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0


In [9]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

- 데이터에 대해 범주형과 숫자형이 복합적으로 이루어져 있습니다.
    - 각 변수들의 변수형이 알맞게 이루어져 있는지 확인할 필요가 있습니다.

In [10]:
dt.isna().sum()

시군구                             0
번지                            225
본번                             75
부번                             75
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1112839
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         870691
k-전화번호                     870274
k-팩스번호                     872742
단지소개기존clob                1050240
k-세대타입(분양형태)               869563
k-관리방식                     869563
k-복도유형                     869890
k-난방방식                     869563
k-전체동수                     870630
k-전체세대수                    869563
k-건설사(시공사)                 871058
k-시행사                      871254
k-사용검사일-사용승인일              869696
k-연면적                      869563
k-주거전용면적      

- 결측치가 대부분 많이 존재합니다.
- 흔하게 구할 수 있는 위도, 경도 좌표를 이용하면 좋을것 같습니다.

In [11]:
dt.describe()

,본번,부번,전용면적(㎡),계약년월,계약일,층,건축년도,해제사유발생일,단지소개기존clob,k-전체동수,...,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,건축면적,주차대수,좌표X,좌표Y,target
count,1.118747e+06,1.118747e+06,1.118822e+06,1.118822e+06,1.118822e+06,1.118822e+06,1.118822e+06,5.983000e+03,68582.000000,248192.000000,...,249259.000000,249214.000000,249214.000000,249214.000000,327.0,2.491080e+05,249108.000000,249152.000000,249152.000000,1.118822e+06
mean,5.649108e+02,5.978885e+00,7.717475e+01,2.014760e+05,1.580656e+01,8.871968e+00,1.998755e+03,2.021057e+07,541.529979,14.798346,...,120726.487549,477.912838,476.713439,167.528470,70.0,1.895070e+05,1063.678778,126.995228,37.545785,5.799153e+04
std,5.160642e+02,4.668584e+01,2.936423e+01,4.187868e+02,8.721166e+00,5.982584e+00,9.333908e+00,1.060697e+04,751.809853,17.693533,...,129020.276480,759.909400,727.553569,248.928143,0.0,1.729027e+06,1235.437604,0.091045,0.052483,4.642602e+04
min,0.000000e+00,0.000000e+00,1.002000e+01,2.007010e+05,1.000000e+00,-4.000000e+00,1.961000e+03,2.020022e+07,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,70.0,0.000000e+00,0.000000,126.798318,37.447843,3.500000e+02
25%,1.760000e+02,0.000000e+00,5.965000e+01,2.011100e+05,8.000000e+00,4.000000e+00,1.992000e+03,2.020082e+07,4.000000,5.000000,...,40735.000000,48.000000,95.000000,0.000000,70.0,0.000000e+00,315.000000,126.913157,37.499201,3.050000e+04
50%,4.700000e+02,0.000000e+00,8.188000e+01,2.015070e+05,1.600000e+01,8.000000e+00,2.000000e+03,2.021030e+07,174.000000,10.000000,...,78125.000000,225.000000,256.000000,63.000000,70.0,1.710550e+03,683.000000,127.014971,37.544936,4.480000e+04
75%,7.810000e+02,1.000000e+00,8.496000e+01,2.018040e+05,2.300000e+01,1.200000e+01,2.005000e+03,2.022021e+07,725.000000,17.000000,...,159544.000000,576.000000,582.000000,237.000000,70.0,8.414210e+03,1274.000000,127.059590,37.577117,6.980000e+04
max,4.974000e+03,2.837000e+03,4.243200e+02,2.023060e+05,3.100000e+01,6.900000e+01,2.023000e+03,2.023093e+07,2888.000000,124.000000,...,969877.000000,4975.000000,5132.000000,1500.000000,70.0,3.159620e+07,12096.000000,127.179998,37.687725,1.450000e+06


## 3. Data Imputation
- 위도, 경도를 채울 수 있는 방법을 활용하기 위해 geopy를 활용하여 좌표를 채우기로 결정하였다.
- 좌표를 가지고 있게되면 할 수 있는 것이 다양하게 있을 것이라는 기대로 진행하였다.
    - 버스 정거장과 아파트와의 거리
    - 아파트에서 지하철까지 떨어져 있는 거리
    - 한강으로부터 떨어져 있는 아파트의 거리  
    ...

- 아파트의 가격에 영향을 미칠 수 있는 거리에 관련된 변수를 모두 생성할 수 있기 때문에 미리 좌표를 구한다.


### 3-1. 지번주소 생성하기

In [12]:
# 복제본으로 해보자
train = dt.copy()
test = dt_test.copy()

- 원본 데이터의 변형을 막기 위해 별도 이름으로 다시 불러오기
- 좌표를 찾기 위해 지번 주소 또는 도로명 주소를 생성하고자 한다.
- 시군구 + 번지를 통합하면 지번주소가 나오기 때문에 지번 주소로 통합하려 한다.

In [15]:
# 시군구, 번지가 결측치인 값을 찾아보자
train[['시군구', '번지']].isna().sum()

시군구      0
번지     225
dtype: int64

- 번지수가 결측치인 값이 존재한다. 해당하는 아파트의 이름을 찾아본다.

In [16]:
# 번지가 결측치인 아파트명 찾기
train['아파트명'][train['번지'].isna()].unique()

array(['서초포레스타2단지', '힐스테이트 서초 젠트리스'], dtype=object)

- 두개의 아파트만 번지수를 결측치로 갖고 있었기 때문에 단순 검색으로 찾아서 대입해주자.

In [17]:
# 지번주소 번지수 없는 놈
train.loc[(train['시군구'] == '서울특별시 서초구 신원동') & (train['아파트명'] == '힐스테이트 서초 젠트리스'), '번지'] = '557'
train.loc[(train['시군구'] == '서울특별시 서초구 내곡동') & (train['아파트명'] == '서초포레스타2단지'), '번지'] = '143'

test.loc[(test['시군구'] == '서울특별시 서초구 신원동') & (test['아파트명'] == '힐스테이트 서초 젠트리스'), '번지'] = '557'
test.loc[(test['시군구'] == '서울특별시 서초구 내곡동') & (test['아파트명'] == '서초포레스타2단지'), '번지'] = '143'

In [18]:
# 지번주소 생성
train['지번주소'] = train['시군구']+' '+train['번지']
test['지번주소'] = test['시군구']+' '+test['번지']

- 이로써 지번주소 변수는 결측치 없이 잘 생성되었다.

### 3-2. geopy를 활용해 지번주소로 검색하여 위도 및 경도 좌표 찾기

In [20]:
# 지번주소를 활용해서 위도 경도 채우기 위한 도구 -> 서울 아파트 집값이기 때문에 한국으로 한정하여 검색
geo_local = Nominatim(user_agent='South Korea')

In [21]:
# 있으면 위경도, 없으면 0, 0 뽑아내주는 로직으로 짬
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [23]:
# sampling
geo_local.geocode(["서울특별시 강남구 개포동 652"])

Location(개포동, 개포2동, 강남구, 서울특별시, 06331, 대한민국, (37.48421, 127.07006, 0.0))

> 한 개의 sample을 활용해 값이 제대로 나오는지 테스트를 적용해보았다.  
> 값이 잘 나오는 것을 확인할 수 있었다.

- 111만 8822개의 모든 칼럼을 전부 검색하는 것 보다 겹치는 값이 있는 좌표는 한번만 조사하도록 진행한다.
- address라는 변수를 만들어 정보를 저장하고 train 및 test에 merge 시키자

In [62]:
# 주소파악을 위해 필요한 값들
address = train[['지번주소', '좌표X', '좌표Y']].drop_duplicates().reset_index()[['지번주소', '좌표X', '좌표Y']]
address

,지번주소,좌표X,좌표Y
0,서울특별시 강남구 개포동 658-1,127.057210,37.476763
1,서울특별시 강남구 개포동 652,127.055990,37.483894
2,서울특별시 강남구 개포동 12-2,127.076624,37.496296
3,서울특별시 강남구 개포동 141,127.058521,37.480002
4,서울특별시 강남구 개포동 187,127.068028,37.487802
...,...,...,...
8944,서울특별시 구로구 구로동 794-32,NaN,NaN
8945,서울특별시 구로구 구로동 807-39,NaN,NaN
8946,서울특별시 서초구 반포동 16-1,NaN,NaN
8947,서울특별시 서초구 서초동 1686-4,NaN,NaN


- 좌표가 결측치인 값도 있지만 결측치 값이 아닌 해당 좌표가 채워져 있는 열도 존재

In [20]:
# geocode unique values
#for i, add in enumerate(address['지번주소']):
#    if address['좌표X'].isna()[i]:
#        print( f"{i}번째 변환중입니다.")
#        address.loc[i, ['좌표Y', '좌표X']] = geocoding(add)

- 좌표가 있는 값은 패스하고 없는 값을 geocoding함수를 통해 값을 채워넣자

In [22]:
# 일단 저장해 저장 무조건 저장
# address.to_csv('../address_to_latlon.csv', index = False)

- 자료가 손실되지 않기 위해 순간마다 바로 저장

In [52]:
### 좌표 불러오기
# address2 = pd.read_csv('../address_to_latlon.csv')

- 못찾은 좌표 : [0.0, 0.0] 으로 이루어져 있는 값 확인

In [53]:
# 못찾은 좌표 확인
# address2[address2['좌표X'] == 0]

### 99개의 좌표 -> 노가다나 규칙을 찾아서 대입해보자

,지번주소,좌표X,좌표Y
111,서울특별시 강남구 대치동 889-74,0.0,0.0
922,서울특별시 강북구 번동 148-496,0.0,0.0
923,서울특별시 강북구 번동 148-500,0.0,0.0
924,서울특별시 강북구 번동 148-503,0.0,0.0
930,서울특별시 강북구 번동 148-494,0.0,0.0
...,...,...,...
8447,서울특별시 은평구 대조동 14-320,0.0,0.0
8448,서울특별시 은평구 불광동 484-57,0.0,0.0
8532,서울특별시 구로구 구로동 749-96,0.0,0.0
8669,서울특별시 동대문구 이문동 257-516,0.0,0.0


- 부 지번이 존재하는 경우 검색이 되지 않는 경우도 존재하기 때문에 이를 없애고 다시한번 진행

In [54]:
# 값이 없는 지번주소를 채우기 위해 지번주소에서 부지번 제외
# address2['새지번주소'] = address2['지번주소'].str.split('-', expand = True)[0]

In [55]:
# # geocode unique values
# for i, add in enumerate(address2['새지번주소']):
#     if address2['좌표X'][i] == 0:
#         print( f"{i}번째 변환중입니다.")
#         address2.loc[i, ['좌표Y', '좌표X']] = geocoding(add)

111번째 변환중입니다.


922번째 변환중입니다.
923번째 변환중입니다.
924번째 변환중입니다.
930번째 변환중입니다.
933번째 변환중입니다.
1484번째 변환중입니다.
1531번째 변환중입니다.
1620번째 변환중입니다.
1633번째 변환중입니다.
1640번째 변환중입니다.
1665번째 변환중입니다.
1696번째 변환중입니다.
1921번째 변환중입니다.
2250번째 변환중입니다.
2351번째 변환중입니다.
2598번째 변환중입니다.
2652번째 변환중입니다.
2653번째 변환중입니다.
2965번째 변환중입니다.
2989번째 변환중입니다.
3016번째 변환중입니다.
3067번째 변환중입니다.
3085번째 변환중입니다.
3093번째 변환중입니다.
3281번째 변환중입니다.
3437번째 변환중입니다.
3507번째 변환중입니다.
3516번째 변환중입니다.
3522번째 변환중입니다.
3537번째 변환중입니다.
3711번째 변환중입니다.
4165번째 변환중입니다.
4335번째 변환중입니다.
4383번째 변환중입니다.
4387번째 변환중입니다.
4393번째 변환중입니다.
4394번째 변환중입니다.
5271번째 변환중입니다.
5272번째 변환중입니다.
5273번째 변환중입니다.
5274번째 변환중입니다.
5275번째 변환중입니다.
5276번째 변환중입니다.
5277번째 변환중입니다.
5278번째 변환중입니다.
5279번째 변환중입니다.
5280번째 변환중입니다.
5281번째 변환중입니다.
5282번째 변환중입니다.
5283번째 변환중입니다.
5284번째 변환중입니다.
5285번째 변환중입니다.
5286번째 변환중입니다.
5287번째 변환중입니다.
5288번째 변환중입니다.
5289번째 변환중입니다.
5305번째 변환중입니다.
5306번째 변환중입니다.
5325번째 변환중입니다.
5326번째 변환중입니다.
5454번째 변환중입니다.
5459번째 변환중입니다.
5542번째 변환중입니다.
5590번째 변환중입니다.
5776번째 변환중입니다.
5913번째 변환중입니다.
5938번째 변환중입니다.


- 두번째 변환 과정에서도 못찾은 좌표를 확인
- 이 값은 직접 채워넣기로 결정

In [56]:
# 못찾은 좌표 확인
# address2[address2['좌표X'] == 0]

,지번주소,좌표X,좌표Y,새지번주소
924,서울특별시 강북구 번동 148-503,0.0,0.0,서울특별시 강북구 번동 148
1633,서울특별시 관악구 신림동 산28-9,0.0,0.0,서울특별시 관악구 신림동 산28
2250,서울특별시 금천구 시흥동 산173-8,0.0,0.0,서울특별시 금천구 시흥동 산173
2598,서울특별시 도봉구 쌍문동 산69-1,0.0,0.0,서울특별시 도봉구 쌍문동 산69
3016,서울특별시 동작구 사당동 산25-2,0.0,0.0,서울특별시 동작구 사당동 산25
3516,서울특별시 서대문구 홍은동 산11-244,0.0,0.0,서울특별시 서대문구 홍은동 산11
5271,서울특별시 영등포구 여의도동 21-2,0.0,0.0,서울특별시 영등포구 여의도동 21
5272,서울특별시 영등포구 여의도동 28,0.0,0.0,서울특별시 영등포구 여의도동 28
5273,서울특별시 영등포구 여의도동 38-1,0.0,0.0,서울특별시 영등포구 여의도동 38
5274,서울특별시 영등포구 여의도동 41,0.0,0.0,서울특별시 영등포구 여의도동 41


In [57]:
# 못찾은 좌표 확인
# address2[address2['좌표X'] == 0].index

Int64Index([ 924, 1633, 2250, 2598, 3016, 3516, 5271, 5272, 5273, 5274, 5275,
            5276, 5277, 5278, 5279, 5280, 5281, 5282, 5283, 5284, 5285, 5286,
            5287, 5288, 5289, 5305, 5306, 5325, 5326, 6541, 6862, 6863, 8141],
           dtype='int64')

In [58]:
# # 모든 값 대입시키기
# for i, [y, x] in zip(address2[address2['좌표X'] == 0].index, [[37.46448, 126.9406],
# [37.45120, 126.8980],
# [37.65730, 127.0289],
# [37.48822, 126.9695],
# [37.59660, 126.9370],
# [37.52650, 126.9299],
# [37.52077, 126.9208],
# [37.51997, 126.9221],
# [37.52225, 126.9335],
# [37.51741, 126.9271],
# [37.51754, 126.9324],
# [37.51897, 126.9390],
# [37.51954, 126.9385],
# [37.51857, 126.9396],
# [37.52597, 126.9327],
# [37.52044, 126.9240],
# [37.5249356, 126.9320978],
# [37.5197265, 126.9342527],
# [37.5230737, 126.9308342],
# [37.530836, 126.923937],
# [37.520693, 126.9371057],
# [37.5187489, 126.9352064],
# [37.5234314, 126.9342505],
# [37.51873399999999, 126.9327266],
# [37.5214745, 126.9325037],
# [37.5221524, 126.9355586],
# [37.5242554, 126.9112748],
# [37.5257944, 126.9087898],
# [37.4808353, 126.8398192],
# [37.524096, 126.9108051],
# [37.52606, 126.9099561],
# [37.5241602, 126.9098694]]):
    
#     address2.loc[i, ['좌표X', '좌표Y']] = [x, y]

In [60]:
# 일단 저장해 저장 무조건 저장
#address2 = address2.drop([22, 33, 6277, 6331, 6399, 8929]).reset_index()
#address2[['지번주소', '좌표X', '좌표Y']].to_csv('../address_to_latlon_complete.csv', index = False)

- 위에서 대입한 값이 제대로 안붙었기 때문에 파일로 만들어서 다시 붙이기로 결정
- '검색해도안나오는좌표.csv'를 별도로 만들어서 붙여넣었다.

In [101]:
# load complete data
a = pd.read_csv('../data/address_to_latlon_complete.csv')
noa = pd.read_csv('../data/검색해도안나오는좌표.csv')

In [102]:
# 중복되거나 붙일때 이상하게 붙은 값 전부 제거
a2 = a.drop_duplicates(['지번주소'])

In [106]:
# 검색해도 안나오는 좌표 가져오기
names = a2[a2['지번주소'].isin(noa['지번주소'])]['지번주소']

In [110]:
# 총 개수가 같은지 확인
noa

,지번주소,좌표Y,좌표X
0,서울특별시 강북구 번동 148-503,37.624327,127.037714
1,서울특별시 관악구 신림동 산28-9,37.464226,126.940686
2,서울특별시 금천구 시흥동 산173-8,37.450915,126.897925
3,서울특별시 도봉구 쌍문동 산69-1,37.657287,127.028954
4,서울특별시 동작구 사당동 산25-2,37.488305,126.969453
5,서울특별시 서대문구 홍은동 산11-244,37.596415,126.936841
6,서울특별시 영등포구 여의도동 21-2,37.526508,126.929838
7,서울특별시 영등포구 여의도동 28,37.520749,126.920802
8,서울특별시 영등포구 여의도동 38-1,37.519892,126.922114
9,서울특별시 영등포구 여의도동 41,37.522142,126.933684


- 이름 같은 값에 적용하여 추가하기

In [128]:
# 방법 바꿔서 저장하기
for n in names:
    a2.loc[a2['지번주소'] == n, ['좌표X뉴']] = noa[noa['지번주소'] == n]['좌표X'].values
    a2.loc[a2['지번주소'] == n, ['좌표Y뉴']] = noa[noa['지번주소'] == n]['좌표Y'].values

In [151]:
# 저장
a2.to_csv('../data/address_to_latlon_complete.csv', index = False)

In [152]:
# load complete data
a = pd.read_csv('../data/address_to_latlon_complete.csv')

- 최종적으로 만든 지번주소와 좌표데이터 합쳐서 결측치인 값 없게끔 merge를 시켜서 반영
- 그럼에도 testset에서 반영이 되지 않는 좌표가 있음 -> train에 존재하지 않는 데이터이면서 좌표가 비어있는 아파트일 가능성이 높음

In [143]:
# train, test에 위경도 좌표 결측 채워두기
complete_latlon_train = pd.merge(train, a, how = 'left', on = '지번주소')
complete_latlon_test = pd.merge(test, a, how = 'left', on = '지번주소')

# train 확인
complete_latlon_train.isna().sum()

시군구                             0
번지                              0
본번                             75
부번                             75
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1116905
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         874774
k-전화번호                     874357
k-팩스번호                     876825
단지소개기존clob                1054323
k-세대타입(분양형태)               873646
k-관리방식                     873646
k-복도유형                     873973
k-난방방식                     873646
k-전체동수                     874713
k-전체세대수                    873646
k-건설사(시공사)                 875141
k-시행사                      875337
k-사용검사일-사용승인일              873779
k-연면적                      873646
k-주거전용면적      

In [145]:
# test 확인
complete_latlon_test.isna().sum()

시군구                          0
번지                           0
본번                           0
부번                           0
아파트명                        10
전용면적(㎡)                      0
계약년월                         0
계약일                          0
층                            0
건축년도                         0
도로명                          0
해제사유발생일                   9098
등기신청일자                       0
거래유형                         0
중개사소재지                       0
k-단지분류(아파트,주상복합등등)        6620
k-전화번호                    6614
k-팩스번호                    6644
단지소개기존clob                8756
k-세대타입(분양형태)              6600
k-관리방식                    6600
k-복도유형                    6602
k-난방방식                    6600
k-전체동수                    6615
k-전체세대수                   6600
k-건설사(시공사)                6617
k-시행사                     6618
k-사용검사일-사용승인일             6601
k-연면적                     6600
k-주거전용면적                  6600
k-관리비부과면적                 6600
k-전용면적별세대현황(60㎡이하)        6600
k-전용면적별세

- 없는 값을 직접 채워넣는 것은 data leakage를 발생 시킬 수 있음
- 따라서 geopy를 활용해 일단 검색을 하고 없는 값은 시군구의 중심 좌표를 반영해서 진행하기로 함

In [137]:
# test데이터에서 address에 아직 없는것
# b = complete_latlon_test[complete_latlon_test['좌표X뉴'].isna()][['지번주소', '좌표X뉴', '좌표Y뉴']].drop_duplicates('지번주소').reset_index()

In [138]:
# geocode unique values for testset
# for i, add in enumerate(b['지번주소']):
#     print( f"{i}번째 변환중입니다.")
#     y, x = geocoding(add)
#     b.loc[i, ['좌표Y뉴', '좌표X뉴']] = [y, x]

0번째 변환중입니다.
1번째 변환중입니다.
2번째 변환중입니다.
3번째 변환중입니다.
4번째 변환중입니다.
5번째 변환중입니다.
6번째 변환중입니다.
7번째 변환중입니다.
8번째 변환중입니다.
9번째 변환중입니다.
10번째 변환중입니다.
11번째 변환중입니다.
12번째 변환중입니다.
13번째 변환중입니다.
14번째 변환중입니다.
15번째 변환중입니다.
16번째 변환중입니다.


- 혹시 나중에 필요할 수도 있기 때문에 찾은 address 전부 묶어서 따로 저장해두자

In [139]:
# c = pd.concat([a, b[['지번주소', '좌표X뉴', '좌표Y뉴']]], axis = 0).reset_index()[['지번주소', '좌표X뉴', '좌표Y뉴']]

In [60]:
# 따로 address파일로 합치는게 나을거같다라는 생각이 들기 때문에 train + test 없는거 다 추가해서 저장
# c.to_csv('../data/address_to_latlon_complete.csv', index = False)

In [147]:
# 잘 들어갔는지 확인
# complete_latlon_test.isna().sum()

- 최종적으로 다 찾은 좌표 데이터에 sample로 반영해보기

In [93]:
# 생성 변수 제거
complete_latlon_train[['좌표X', '좌표Y']] = complete_latlon_train[['좌표X뉴', '좌표Y뉴']]
cltrain = complete_latlon_train.drop(['좌표X뉴', '좌표Y뉴', '지번주소'], axis = 1)

complete_latlon_test[['좌표X', '좌표Y']] = complete_latlon_test[['좌표X뉴', '좌표Y뉴']]
cltest = complete_latlon_test.drop(['좌표X뉴', '좌표Y뉴', '지번주소'], axis = 1)

In [94]:
complete_latlon_test

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,지번주소,좌표X뉴,좌표Y뉴
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,서울특별시 강남구 개포동 658-1,127.057210,37.476763
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,서울특별시 강남구 개포동 651-1,127.056394,37.484892
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.055990,37.483894
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.055990,37.483894
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,의무,1984-12-22 00:00:00.0,Y,N,127.055990,37.483894,2013-03-07 09:46:28.0,서울특별시 강남구 개포동 652,127.055990,37.483894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106720,37.618870
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106720,37.618870
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106720,37.618870
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2014,...,의무,2015-09-09 15:30:27.0,Y,N,127.106720,37.618870,2014-09-01 13:05:03.0,서울특별시 중랑구 신내동 816,127.106720,37.618870


In [95]:
# 일단 저장해 저장 무조건 저장
cltest.to_csv('../data/test_latlon_complete.csv', index = False)
cltrain.to_csv('../data/train_latlon_complete.csv', index = False)